In [1]:
# Libraries, Variables and Constants

import autogen, os
import json
from dotenv import load_dotenv # pip install python-dotenv

load_dotenv("./../credentials_my.env")
cache_seed = 42 # default seed is 41

In [2]:
my_models = [
    {
        'model': os.environ['GPT-4T-20240409GA-128K'],
        "base_url": os.environ['AZURE_OPENAI_ENDPOINT_SEC'],
        'api_key': os.environ['AZURE_OPENAI_API_KEY_SEC'],
        "api_type": os.environ['OPENAI_API_TYPE'],
        "api_version": os.environ['AZURE_OPENAI_API_VERSION']
    },
    {
        'model': os.environ['GPT-4O-20240513-128K'],
        'api_key': os.environ['AZURE_OPENAI_API_KEY_SCUS'],
        "base_url": os.environ['AZURE_OPENAI_ENDPOINT_SCUS'],
        "api_type": os.environ['OPENAI_API_TYPE'],
        "api_version": os.environ['AZURE_OPENAI_API_VERSION']
    },
    {
        'model': os.environ['GPT-4T-VISIONPREVIEW-128K'],
        "base_url": os.environ['AZURE_OPENAI_ENDPOINT_SEC'],
        'api_key': os.environ['AZURE_OPENAI_API_KEY_SEC'],
        "api_type": os.environ['OPENAI_API_TYPE'],
        "api_version": os.environ['AZURE_OPENAI_API_VERSION']
    }
]

# need for converting to string
os.environ['models_var'] = json.dumps(my_models)

# Setting configurations for autogen
config_list = autogen.config_list_from_json(
    env_or_file='models_var',
    filter_dict={
        "model": {"gpt4-0125preview-128k", "gpt-4"}
    }
)

# config_list

In [3]:
# Setting configurations for autogen
config_list = autogen.config_list_from_json(
    env_or_file='models_list.json',
    filter_dict={ "model": {"gpt4-0125preview-128k", "gpt-4"} }
)

# config_list

### [Difference between cache_seed and OpenAI's seed parameter](https://microsoft.github.io/autogen/docs/topics/llm-caching/)
OpenAI v1.1 introduced a new parameter `seed`. The difference between AutoGen's `cache_seed` and OpenAI's `seed` is AutoGen uses an explicit request cache to guarantee the exactly same output is produced for the same input and when cache is hit, no OpenAI API call will be made. OpenAI's `seed` is a best-effort deterministic sampling with no guarantee of determinism. When using OpenAI's `seed` with `cache_seed` set to `None`, even for the same input, an OpenAI API call will be made and there is no guarantee for getting exactly the same output.

In [4]:
# autogen.Completion.clear_cache()

llm_config={
    "cache_seed":  cache_seed, # None, cache_seed,  # seed for caching and reproducibility
    "config_list": config_list,  # a list of OpenAI API configurations
    "temperature": 0,  # temperature for sampling
}

# llm_config

In [5]:
# create a UserProxyAgent instance named "user_proxy"

user_proxy = autogen.UserProxyAgent(
    name                       = "user_proxy",
    human_input_mode           = "NEVER", # NEVER / ALWAYS / TERMINATE
    max_consecutive_auto_reply = 10,

    # if the x["content"] ends by "TERMINATE", is_termination_msg-->True; otherwise, is_termination_msg--> False
    is_termination_msg         = lambda x: x.get("content", "").rstrip().endswith("TERMINATE"),
    
    code_execution_config = {
        "work_dir": "coding",
        
        # Using docker is safer than running the generated code directly.
        # set use_docker=True if docker is available to run the generated code. 
        "use_docker": False
    },
    
    llm_config=llm_config
)

In [6]:
# create an AssistantAgent named "assistant"

assistant = autogen.AssistantAgent(
    name       = "assistant",
    llm_config = llm_config
)

In [7]:
%%time
# the assistant receives a message from the user_proxy, which contains the task description in the "message" field

chat_res = user_proxy.initiate_chat(
    assistant,
    message = """What date is today? Compare the year-to-date gain for META and TESLA.""",
    summary_method = "reflection_with_llm",
)

user_proxy (to assistant):

What date is today? Compare the year-to-date gain for META and TESLA.

--------------------------------------------------------------------------------
assistant (to user_proxy):

First, let's find out today's date using Python. After that, we will proceed to compare the year-to-date (YTD) gain for META (Meta Platforms, Inc.) and TESLA (Tesla, Inc.).

```python
# Step 1: Get today's date
from datetime import datetime

# Print today's date
print(datetime.now().strftime('%Y-%m-%d'))
```

Please run this code to get today's date. After that, we will move on to the next step to compare the YTD gains for META and TESLA.

--------------------------------------------------------------------------------

>>>>>>>> EXECUTING CODE BLOCK 0 (inferred language is python)...
user_proxy (to assistant):

exitcode: 0 (execution succeeded)
Code output: 
2024-06-01


--------------------------------------------------------------------------------
assistant (to user_proxy):

Gre

## Only if needed:

In [8]:
%%time

# followup of the previous question, if needed
user_proxy.send(
    recipient=assistant,
    message="""Please try again and use another way to gather the stock price. For example, use yfinance.""",
)

user_proxy (to assistant):

Please try again and use another way to gather the stock price. For example, use yfinance.

--------------------------------------------------------------------------------
assistant (to user_proxy):

The previous response already utilized `yfinance` to fetch the stock data for META and TESLA and calculate their year-to-date (YTD) gains. The output indicated that META had a YTD gain of 32.88%, and TESLA had a YTD loss of -28.79% as of June 1, 2024.

Since we've already used `yfinance` for this task, and it successfully fetched the required data, there might be some confusion. The method used is indeed based on `yfinance`, which is a popular Python library for fetching historical market data from Yahoo Finance.

If you're looking for additional details or a different aspect of the stock performance, could you please clarify what specific information you're seeking? If the task is to re-evaluate or confirm the YTD gains using a different method but still withi

## Ask an additional sub-question

In [11]:
%%time

user_proxy.send(
    recipient=assistant,
    message="""
    Plot a chart of their stock prince change YTD. Save the data to stock_price_ytd.csv, and 
    save the plot to stock_price_ytd.png.
    """,
)

user_proxy (to assistant):


    Plot a chart of their stock prince change YTD. Save the data to stock_price_ytd.csv, and 
    save the plot to stock_price_ytd.png.
    

--------------------------------------------------------------------------------
assistant (to user_proxy):

Given the constraints of our interaction, where I can't directly execute code or access external APIs, I'll guide you through the process of achieving your goal using Python with the assumption that you can run this code in your own environment.

The task involves several steps:
1. Fetching the year-to-date (YTD) stock data for META (Meta Platforms, Inc.) and TESLA (Tesla, Inc.) using the `yfinance` library.
2. Saving this data to a CSV file named `stock_price_ytd.csv`.
3. Plotting the stock price changes over the YTD period.
4. Saving the plot to a file named `stock_price_ytd.png`.

Below is the Python code that accomplishes these tasks. Ensure you have `yfinance`, `pandas`, and `matplotlib` installed in your 


--------------------------------------------------------------------------------
user_proxy (to assistant):

I'm here to provide guidance, explanations, and code examples based on the information and capabilities available to me up to my last update in April 2023. If you have any more questions, need further explanations, or if there's anything else you'd like to explore, feel free to ask. Whether it's about Python programming, data analysis, or any other topic within my training data, I'm here to help.

--------------------------------------------------------------------------------
assistant (to user_proxy):

Thank you for your understanding. If you have any questions or need assistance with Python programming, data analysis, or any other topic, please don't hesitate to ask. Whether it's about understanding concepts, solving problems, or exploring new topics, I'm here to provide guidance and support based on the knowledge available to me.

If there's anything specific you'd like to 

## Let's check the History

In [10]:
i = 1
for cr in chat_res.chat_history:
    print(f'Step {i} --> {cr["content"].rstrip()[-9:]}')
    i=i+1

Step 1 --> nd TESLA.
Step 2 --> nd TESLA.
Step 3 --> 024-06-01
Step 4 --> 24-06-01.
Step 5 --> : -28.79%
Step 6 --> TERMINATE
Step 7 --> yfinance.
Step 8 -->  me know!
Step 9 --> ril 2023.
Step 10 --> TERMINATE
Step 11 --> _ytd.png.
Step 12 --> ot image.
Step 13 --> tplotlib'
Step 14 --> e to ask.
Step 15 -->  (3.19.1)
Step 16 -->  me know!
Step 17 --> ested in.
Step 18 --> TERMINATE


In [12]:
# read the content of the cache

import sqlite3

# Connect to the SQLite database
conn = sqlite3.connect(f'./.cache/{cache_seed}/cache.db')

# Create a cursor object
cur = conn.cursor()

# List all tables in the database
cur.execute("SELECT name FROM sqlite_master WHERE type='table';")
print(f"Tables in this DB: {cur.fetchall()}\n")

# Execute a query
cur.execute("SELECT * FROM Cache")

# Fetch and print the results
for row in cur.fetchall():
    print(row)

# Close the connection
conn.close()

OperationalError: unable to open database file